In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                        dcc.Dropdown(id='site-dropdown',options = [
                           {'label': 'CCAFS LC-40', 'value': 'OPT1'},
                           {'label': 'KSCLC-39A', 'value': 'OPT2'},
                           {'label': 'VAFB SLC-4E', 'value': 'OPT3'},
                           {'label': 'CCAFS SLC-40', 'value': 'OPT4'},
                           {'labeL': 'ALL', 'value': 'OPT5'},
                           ],
                value = all,
                  placeholder='Select a Launch Site here',
                  searchable = True),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min = 0,max = 10000, step = 1000,value = min_payload and max_payload),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))

# Add computation to callback function and return graph
def get_graph(entered_site):
    if entered_site == 'ALL':
        df = spacex_df
        v_counts = df['class'].value_counts()
    else:
        df =  spacex_df[spacex_df['Launch Site']==(entered_site)]
        v_counts = df['class'].value_counts()
    fig = px.pie(df, values=v_counts, names='class', title='Success Rate')
    return fig
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input([Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]))
def get_graph2(entered_site):
    if entered_site == 'ALL':
        df = spacex_df

    else:
        df =  spacex_df[spacex_df['Launch Site']==(entered_site)]

    fig = go.Figure(data=go.Scatter(x=df['Payload Mass (kg)'], y=df['class'], mode='lines', marker=dict(color="Booster Version Category")))
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()